In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np
import statsmodels.api as sm
import hockey_scraper
import pickle
import time
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import normalize, FunctionTransformer
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import log_loss
from scipy import stats
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_roc_curve
from sklearn.metrics import confusion_matrix, plot_confusion_matrix,\
    precision_score, recall_score, accuracy_score, f1_score, log_loss,\
    roc_curve, roc_auc_score, classification_report
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier, AdaBoostRegressor, GradientBoostingClassifier
from collections import Counter
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, GridSearchCV

from sklearn.pipeline import make_pipeline, Pipeline
pd.set_option('display.max_columns', None)

The purpose of this notebook is to help with feature selection. Specifically I have calculate team stats using 3, 5, 10, 20, and 30 game rolling windows. I want to determine which window or mix results in the best model. To do so I will first train a simple logistic regression model using each rolling windo specifically. Then I will try Scikit-learn's select K best to see one comes out from that. 

In [2]:
df = pd.read_csv('data/all_games_multirolling_noSVA.csv')

In [3]:
df.head()

,Unnamed: 0,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Last_20_FenwickSV%,home_Last_20_GSAx/60,home_Last_20_HDCSV%,away_goalie,away_Last_20_FenwickSV%,away_Last_20_GSAx/60,away_Last_20_HDCSV%,home_Team_Key,home_last_3_FF%_5v5,home_last_3_GF%_5v5,home_last_3_xGF%_5v5,home_last_3_SH%,home_last3_pp_TOI_per_game,home_last3_xGF_per_min_pp,home_last3_pk_TOI_per_game,home_last3_xGA_per_min_pk,home_B2B,home_last_5_FF%_5v5,home_last_5_GF%_5v5,home_last_5_xGF%_5v5,home_last_5_SH%,home_last5_pp_TOI_per_game,home_last5_xGF_per_min_pp,home_last5_pk_TOI_per_game,home_last5_xGA_per_min_pk,home_last_10_FF%_5v5,home_last_10_GF%_5v5,home_last_10_xGF%_5v5,home_last_10_SH%,home_last10_pp_TOI_per_game,home_last10_xGF_per_min_pp,home_last10_pk_TOI_per_game,home_last10_xGA_per_min_pk,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_last_30_FF%_5v5,home_last_30_GF%_5v5,home_last_30_xGF%_5v5,home_last_30_SH%,home_last30_pp_TOI_per_game,home_last30_xGF_per_min_pp,home_last30_pk_TOI_per_game,home_last30_xGA_per_min_pk,away_Team_Key,away_last_3_FF%_5v5,away_last_3_GF%_5v5,away_last_3_xGF%_5v5,away_last_3_SH%,away_last3_pp_TOI_per_game,away_last3_xGF_per_min_pp,away_last3_pk_TOI_per_game,away_last3_xGA_per_min_pk,away_B2B,away_last_5_FF%_5v5,away_last_5_GF%_5v5,away_last_5_xGF%_5v5,away_last_5_SH%,away_last5_pp_TOI_per_game,away_last5_xGF_per_min_pp,away_last5_pk_TOI_per_game,away_last5_xGA_per_min_pk,away_last_10_FF%_5v5,away_last_10_GF%_5v5,away_last_10_xGF%_5v5,away_last_10_SH%,away_last10_pp_TOI_per_game,away_last10_xGF_per_min_pp,away_last10_pk_TOI_per_game,away_last10_xGA_per_min_pk,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_last_30_FF%_5v5,away_last_30_GF%_5v5,away_last_30_xGF%_5v5,away_last_30_SH%,away_last30_pp_TOI_per_game,away_last30_xGF_per_min_pp,away_last30_pk_TOI_per_game,away_last30_xGA_per_min_pk
0,0,2017020001,2017-10-04,Bell MTS Place,WPG,TOR,2017-10-04 23:00:00,2,7,Final,0,WPG_2017-10-04,TOR_2017-10-04,Steve Mason,0.946716,0.259293,0.917127,Frederik Andersen,0.947245,0.043154,0.874317,WPG_2017-10-04,50.222222,60.000000,46.666667,10.112360,3.444444,0.063871,4.005556,0.090707,0.0,52.560647,56.521739,51.729107,9.285714,4.190000,0.079714,3.693333,0.098556,51.240876,50.000000,51.893740,7.228916,5.128333,0.091453,4.315000,0.128158,49.213483,51.190476,50.903164,8.977035,5.860833,0.102801,5.440000,0.105423,49.276086,50.000000,48.848563,8.720113,5.171111,0.119961,4.815000,0.115057,TOR_2017-10-04,53.777778,50.000000,51.673469,8.823529,6.450000,0.106460,3.394444,0.111948,0.0,53.457447,47.368421,51.673469,7.086614,5.893333,0.069910,3.070000,0.074267,50.712251,48.648649,50.580307,7.860262,5.073333,0.080420,4.235000,0.114522,49.103448,48.611111,48.707092,7.526882,4.525833,0.118431,4.195000,0.123361,50.022655,50.925926,50.175861,7.462687,4.538889,0.128519,4.363889,0.133978
1,1,2017020002,2017-10-04,PPG Paints Arena,PIT,STL,2017-10-05 00:00:00,4,5,Final,0,PIT_2017-10-04,STL_2017-10-04,Matt Murray,0.944706,0.457950,0.889831,Jake Allen,0.952147,0.199829,0.898810,PIT_2017-10-04,42.180095,50.000000,46.913580,13.559322,2.761111,0.182294,3.683333,0.171041,0.0,40.960452,57.142857,45.309026,12.121212,3.336667,0.143856,3.546667,0.153383,43.557423,45.238095,43.044560,8.558559,3.633333,0.176422,3.358333,0.113151,47.997139,52.439024,48.360562,8.811475,4.792500,0.134168,4.569167,0.119825,49.062049,52.991453,50.927977,8.310992,4.693889,0.123707,4.812778,0.120859,STL_2017-10-04,43.661972,47.058824,47.543860,10.666667,7.283333,0.113043,5.250000,0.067937,0.0,46.290801,50.000000,50.637826,11.570248,6.000000,0.096000,4.966667,0.109128,50.678733,60.465116,55.805893,10.116732,5.313333

In [4]:
def model_and_evaluate(df, feature_columns, scoring = 'neg_log_loss'):
    df = df.dropna()
    X = df[feature_columns]
    y = df['Home_Team_Won']


    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=2020)

    pipeline = make_pipeline(StandardScaler(), LogisticRegression(random_state=2020,  max_iter=1000))

    return cross_val_score(pipeline, X_train, y_train, cv=5, scoring = scoring).mean()


In [5]:
list(df.columns)

['Unnamed: 0',
 'game_id',
 'date',
 'venue',
 'home_team',
 'away_team',
 'start_time',
 'home_score',
 'away_score',
 'status',
 'Home_Team_Won',
 'Home_Team_Key',
 'Away_Team_Key',
 'home_goalie',
 'home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_goalie',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%',
 'home_Team_Key',
 'home_last_3_FF%_5v5',
 'home_last_3_GF%_5v5',
 'home_last_3_xGF%_5v5',
 'home_last_3_SH%',
 'home_last3_pp_TOI_per_game',
 'home_last3_xGF_per_min_pp',
 'home_last3_pk_TOI_per_game',
 'home_last3_xGA_per_min_pk',
 'home_B2B',
 'home_last_5_FF%_5v5',
 'home_last_5_GF%_5v5',
 'home_last_5_xGF%_5v5',
 'home_last_5_SH%',
 'home_last5_pp_TOI_per_game',
 'home_last5_xGF_per_min_pp',
 'home_last5_pk_TOI_per_game',
 'home_last5_xGA_per_min_pk',
 'home_last_10_FF%_5v5',
 'home_last_10_GF%_5v5',
 'home_last_10_xGF%_5v5',
 'home_last_10_SH%',
 'home_last10_pp_TOI_per_game',
 'home_last10_xGF_per_min_pp',
 'home_la

In [6]:
r3 = ['home_B2B', 'away_B2B', 'home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_last_3_FF%_5v5',
 'home_last_3_GF%_5v5',
 'home_last_3_xGF%_5v5',
 'home_last_3_SH%',
 'home_last3_pp_TOI_per_game',
 'home_last3_xGF_per_min_pp',
 'home_last3_pk_TOI_per_game',
 'home_last3_xGA_per_min_pk', 'away_last_3_FF%_5v5',
 'away_last_3_GF%_5v5',
 'away_last_3_xGF%_5v5',
 'away_last_3_SH%',
 'away_last3_pp_TOI_per_game',
 'away_last3_xGF_per_min_pp',
 'away_last3_pk_TOI_per_game',
 'away_last3_xGA_per_min_pk']
r5 =['home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_B2B', 'away_B2B', 'home_last_5_FF%_5v5',
 'home_last_5_GF%_5v5',
 'home_last_5_xGF%_5v5',
 'home_last_5_SH%',
 'home_last5_pp_TOI_per_game',
 'home_last5_xGF_per_min_pp',
 'home_last5_pk_TOI_per_game',
 'home_last5_xGA_per_min_pk', 'away_last_5_FF%_5v5',
 'away_last_5_GF%_5v5',
 'away_last_5_xGF%_5v5',
 'away_last_5_SH%',
 'away_last5_pp_TOI_per_game',
 'away_last5_xGF_per_min_pp',
 'away_last5_pk_TOI_per_game',
 'away_last5_xGA_per_min_pk']
r10 =['home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_B2B', 'away_B2B', 'home_last_10_FF%_5v5',
 'home_last_10_GF%_5v5',
 'home_last_10_xGF%_5v5',
 'home_last_10_SH%',
 'home_last10_pp_TOI_per_game',
 'home_last10_xGF_per_min_pp',
 'home_last10_pk_TOI_per_game',
 'home_last10_xGA_per_min_pk', 'away_last_10_FF%_5v5',
 'away_last_10_GF%_5v5',
 'away_last_10_xGF%_5v5',
 'away_last_10_SH%',
 'away_last10_pp_TOI_per_game',
 'away_last10_xGF_per_min_pp',
 'away_last10_pk_TOI_per_game',
 'away_last10_xGA_per_min_pk']
r20 = ['home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_B2B', 'away_B2B', 'home_last_20_FF%_5v5',
 'home_last_20_GF%_5v5',
 'home_last_20_xGF%_5v5',
 'home_last_20_SH%',
 'home_last20_pp_TOI_per_game',
 'home_last20_xGF_per_min_pp',
 'home_last20_pk_TOI_per_game',
 'home_last20_xGA_per_min_pk', 'away_last_20_FF%_5v5',
 'away_last_20_GF%_5v5',
 'away_last_20_xGF%_5v5',
 'away_last_20_SH%',
 'away_last20_pp_TOI_per_game',
 'away_last20_xGF_per_min_pp',
 'away_last20_pk_TOI_per_game',
 'away_last20_xGA_per_min_pk']
r30 = ['home_Last_20_FenwickSV%',
 'home_Last_20_GSAx/60',
 'home_Last_20_HDCSV%',
 'away_Last_20_FenwickSV%',
 'away_Last_20_GSAx/60',
 'away_Last_20_HDCSV%', 'home_B2B', 'away_B2B',  'home_last_30_FF%_5v5',
 'home_last_30_GF%_5v5',
 'home_last_30_xGF%_5v5',
 'home_last_30_SH%',
 'home_last30_pp_TOI_per_game',
 'home_last30_xGF_per_min_pp',
 'home_last30_pk_TOI_per_game',
 'home_last30_xGA_per_min_pk', 'away_last_30_FF%_5v5',
 'away_last_30_GF%_5v5',
 'away_last_30_xGF%_5v5',
 'away_last_30_SH%',
 'away_last30_pp_TOI_per_game',
 'away_last30_xGF_per_min_pp',
 'away_last30_pk_TOI_per_game',
 'away_last30_xGA_per_min_pk']

all_r = list(set(r3+r5+r10+r20+r30))

r3_30 =list(set(r3+r30))
r5_30 = list(set(r5+r30))
r10_30 = list(set(r10+r30))
r_3_5_30 = list(set(r3+r5+r30))
r_5_20 = list(set(r5+r20))

In [7]:
windows =  {'3' : r3,'5' : r5,'10' : r10,'20' : r20, '30' : r30, 'all' : all_r, '3+30': r3_30, '5+30': r5_30, '10+30' : r10_30, '3+5+30': r_3_5_30, '5+20': r_5_20}

In [8]:
for k,v in windows.items():
    print(f'{k} rolling cv log loss = {model_and_evaluate(df, v)}')

3 rolling cv log loss = -0.6861562495735097
5 rolling cv log loss = -0.6847321956176653
10 rolling cv log loss = -0.6809962171610652
20 rolling cv log loss = -0.6771632184554629
30 rolling cv log loss = -0.674385744275857
all rolling cv log loss = -0.6862683128580372
3+30 rolling cv log loss = -0.6776354571333767
5+30 rolling cv log loss = -0.6771719023613585
10+30 rolling cv log loss = -0.6788676522172284
3+5+30 rolling cv log loss = -0.6811070794335043
5+20 rolling cv log loss = -0.6807867901085258


In [9]:
for k,v in windows.items():
    print(f'{k} rolling cv accuracy = {model_and_evaluate(df, v, "accuracy")}')

3 rolling cv accuracy = 0.5573552779464389
5 rolling cv accuracy = 0.5579314532277442
10 rolling cv accuracy = 0.5687883524745823
20 rolling cv accuracy = 0.575533651134049
30 rolling cv accuracy = 0.5802321926096262
all rolling cv accuracy = 0.5664423114188638
3+30 rolling cv accuracy = 0.5849285809638232
5+30 rolling cv accuracy = 0.581990000904311
10+30 rolling cv accuracy = 0.5778865821781837
3+5+30 rolling cv accuracy = 0.5737792878335723
5+20 rolling cv accuracy = 0.5687909362202385


## Using Select K Best

In [10]:
df['home_B2B'] = df['home_B2B'].astype('category')
df['away_B2B'] = df['away_B2B'].astype('category')

In [11]:
numerical = [
 'away_last3_xGF_per_min_pp',
 'home_last20_pk_TOI_per_game',
 'away_last_5_SH%',
 'away_last_10_SH%',
 'home_last_10_xGF%_5v5',
 'home_last20_xGF_per_min_pp',
 'home_last_3_xGF%_5v5',
 'home_last3_pp_TOI_per_game',
 'away_last30_xGA_per_min_pk',
 'home_Last_20_HDCSV%',
 'home_last30_pk_TOI_per_game',
 'home_last10_xGF_per_min_pp',
 'away_last_3_xGF%_5v5',
 'home_Last_20_FenwickSV%',
 'away_Last_20_HDCSV%',
 'away_last_30_xGF%_5v5',
 'away_last20_pk_TOI_per_game',
 'home_last30_xGF_per_min_pp',
 'home_last_20_xGF%_5v5',
 'away_last5_xGF_per_min_pp',
 'away_last10_pk_TOI_per_game',
 'home_last_20_SH%',
 'home_last_3_FF%_5v5',
 'home_last10_pk_TOI_per_game',
 'away_last_30_SH%',
 'away_Last_20_GSAx/60',
 'home_last_10_SH%',
 'away_last5_pp_TOI_per_game',
 'home_last_30_GF%_5v5',
 'away_last_3_GF%_5v5',
 'home_last_5_GF%_5v5',
 'away_last30_pp_TOI_per_game',
 'home_last5_pp_TOI_per_game',
 'home_last10_xGA_per_min_pk',
 'away_last_20_GF%_5v5',
 'home_last3_xGA_per_min_pk',
 'home_last_10_GF%_5v5',
 'away_last_5_GF%_5v5',
 'home_last_30_SH%',
 'home_last_30_FF%_5v5',
 'home_Last_20_GSAx/60',
 'home_last5_xGF_per_min_pp',
 'home_last3_pk_TOI_per_game',
 'home_last20_xGA_per_min_pk',
 'away_last_3_SH%',
 'away_last_20_FF%_5v5',
 'away_last30_pk_TOI_per_game',
 'away_last_5_FF%_5v5',
 'home_last20_pp_TOI_per_game',
 'home_last_30_xGF%_5v5',
 'home_last_5_FF%_5v5',
 'home_last_5_xGF%_5v5',
 'away_last_10_xGF%_5v5',
 'home_last_10_FF%_5v5',
 'away_last_3_FF%_5v5',
 'home_last10_pp_TOI_per_game',
 'home_last30_pp_TOI_per_game',
 'home_last_3_SH%',
 'away_last3_pp_TOI_per_game',
 'home_last5_xGA_per_min_pk',
 'away_last_20_SH%',
 'away_last_30_FF%_5v5',
 'away_last10_xGA_per_min_pk',
 'home_last_3_GF%_5v5',
 'away_last30_xGF_per_min_pp',
 'home_last_20_GF%_5v5',
 'away_last5_xGA_per_min_pk',
 'home_last_5_SH%',
 'away_last_20_xGF%_5v5',
 'away_last_5_xGF%_5v5',
 'home_last_20_FF%_5v5',
 'away_Last_20_FenwickSV%',
 'away_last5_pk_TOI_per_game',
 'away_last10_xGF_per_min_pp',
 'away_last3_xGA_per_min_pk',
 'away_last20_pp_TOI_per_game',
 'away_last10_pp_TOI_per_game',
 'away_last_10_GF%_5v5',
 'away_last_30_GF%_5v5',
 'home_last5_pk_TOI_per_game',
 'away_last_10_FF%_5v5',
 'home_last30_xGA_per_min_pk',
 'away_last20_xGF_per_min_pp',
 'away_last20_xGA_per_min_pk',
 'away_last3_pk_TOI_per_game',
 'home_last3_xGF_per_min_pp']

In [13]:

#this code worked i think
pipeline=Pipeline(steps= [
    ('feature_processing', FeatureUnion(transformer_list = [
            ('categorical', FunctionTransformer(lambda data: data.iloc[:, 2:])),

            #numeric
            ('numeric', Pipeline(steps = [
                ('select', FunctionTransformer(lambda data: data.iloc[:, 0:2])),
                ('scale', StandardScaler())
                        ]))
        ])),
    ('skb', SelectKBest(f_classif, k=20))
    ]
)

In [14]:
k_best_df = df.dropna()

X = k_best_df[all_r]
y = k_best_df['Home_Team_Won']

pipeline.fit(X,y)

Pipeline(steps=[('feature_processing',
                 FeatureUnion(transformer_list=[('categorical',
                                                 FunctionTransformer(func=<function <lambda> at 0x7fe351d0b550>)),
                                                ('numeric',
                                                 Pipeline(steps=[('select',
                                                                  FunctionTransformer(func=<function <lambda> at 0x7fe351d0b280>)),
                                                                 ('scale',
                                                                  StandardScaler())]))])),
                ('skb', SelectKBest(k=20))])

In [15]:
cols = pipeline.named_steps['skb'].get_support()
k_best_features = X.iloc[:,cols]

In [16]:
k = [n[9:] for n in k_best_features.columns]

In [17]:
sorted(k)

['',
 '10_xGA_per_min_pk',
 '10_xGF_per_min_pp',
 '30_pp_TOI_per_game',
 '3_pk_TOI_per_game',
 '3_pp_TOI_per_game',
 '5_pk_TOI_per_game',
 '5_xGA_per_min_pk',
 '_10_FF%_5v5',
 '_20_GF%_5v5',
 '_20_GF%_5v5',
 '_20_GSAx/60',
 '_20_SH%',
 '_30_xGF%_5v5',
 '_3_FF%_5v5',
 '_3_FF%_5v5',
 '_3_GF%_5v5',
 '_5_FF%_5v5',
 '_5_SH%',
 '_5_xGF%_5v5']

### Evaluate With Venue and Scoring Adjustments

In [18]:
sva = pd.read_csv('data/all_games_multirolling_SVA.csv')

In [19]:
sva.head()

,Unnamed: 0,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Goalie_FenwickSV%,home_Goalie_GSAx/60,home_Goalie_HDCSV%,away_goalie,away_Goalie_FenwickSV%,away_Goalie_GSAx/60,away_Goalie_HDCSV%,home_Team_Key,home_Game_Number,home_last_3_FF%_5v5,home_last_3_GF%_5v5,home_last_3_xGF%_5v5,home_last_3_SH%,home_last3_pp_TOI_per_game,home_last3_xGF_per_min_pp,home_last3_pk_TOI_per_game,home_last3_xGA_per_min_pk,home_B2B,home_last_5_FF%_5v5,home_last_5_GF%_5v5,home_last_5_xGF%_5v5,home_last_5_SH%,home_last5_pp_TOI_per_game,home_last5_xGF_per_min_pp,home_last5_pk_TOI_per_game,home_last5_xGA_per_min_pk,home_last_10_FF%_5v5,home_last_10_GF%_5v5,home_last_10_xGF%_5v5,home_last_10_SH%,home_last10_pp_TOI_per_game,home_last10_xGF_per_min_pp,home_last10_pk_TOI_per_game,home_last10_xGA_per_min_pk,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_last_30_FF%_5v5,home_last_30_GF%_5v5,home_last_30_xGF%_5v5,home_last_30_SH%,home_last30_pp_TOI_per_game,home_last30_xGF_per_min_pp,home_last30_pk_TOI_per_game,home_last30_xGA_per_min_pk,home_last_40_FF%_5v5,home_last_40_GF%_5v5,home_last_40_xGF%_5v5,home_last_40_SH%,home_last40_pp_TOI_per_game,home_last40_xGF_per_min_pp,home_last40_pk_TOI_per_game,home_last40_xGA_per_min_pk,away_Team_Key,away_Game_Number,away_last_3_FF%_5v5,away_last_3_GF%_5v5,away_last_3_xGF%_5v5,away_last_3_SH%,away_last3_pp_TOI_per_game,away_last3_xGF_per_min_pp,away_last3_pk_TOI_per_game,away_last3_xGA_per_min_pk,away_B2B,away_last_5_FF%_5v5,away_last_5_GF%_5v5,away_last_5_xGF%_5v5,away_last_5_SH%,away_last5_pp_TOI_per_game,away_last5_xGF_per_min_pp,away_last5_pk_TOI_per_game,away_last5_xGA_per_min_pk,away_last_10_FF%_5v5,away_last_10_GF%_5v5,away_last_10_xGF%_5v5,away_last_10_SH%,away_last10_pp_TOI_per_game,away_last10_xGF_per_min_pp,away_last10_pk_TOI_per_game,away_last10_xGA_per_min_pk,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_last_30_FF%_5v5,away_last_30_GF%_5v5,away_last_30_xGF%_5v5,away_last_30_SH%,away_last30_pp_TOI_per_game,away_last30_xGF_per_min_pp,away_last30_pk_TOI_per_game,away_last30_xGA_per_min_pk,away_last_40_FF%_5v5,away_last_40_GF%_5v5,away_last_40_xGF%_5v5,away_last_40_SH%,away_last40_pp_TOI_per_game,away_last40_xGF_per_min_pp,away_last40_pk_TOI_per_game,away_last40_xGA_per_min_pk
0,0,2017020001,2017-10-04,Bell MTS Place,WPG,TOR,2017-10-04 23:00:00,2,7,Final,0,WPG_2017-10-04,TOR_2017-10-04,Steve Mason,0.932657,-0.334940,0.866667,Frederik Andersen,0.942629,0.027934,0.872792,WPG_2017-10-04,1.0,52.073795,62.416999,48.839009,10.258649,3.444444,0.063871,4.005556,0.090707,0.0,52.399869,57.080799,51.663405,9.426112,4.190000,0.079714,3.693333,0.098556,50.977189,50.738779,51.924105,7.380972,5.128333,0.091453,4.315000,0.128158,49.296838,51.281437,51.260619,8.999958,5.860833,0.102801,5.440000,0.105423,49.171944,50.109126,49.007605,8.809652,5.171111,0.119961,4.815000,0.115057,48.803377,50.127801,48.992719,9.025236,5.328333,0.112699,4.923333,0.104858,TOR_2017-10-04,1.0,51.594385,47.355164,48.770492,8.692972,6.450000,0.106460,3.394444,0.111948,0.0,52.562502,45.937500,48.770492,6.967375,5.893333,0.069910,3.070000,0.074267,50.792085,48.572198,49.886878,7.837427,5.073333,0.080420,4.235000,0.114522,49.687136,49.188289,49.131362,7.552033,4.525833,0.118431,4.195000,0.123361,50.085902,51.013795,50.000000,7.493438,4.538889,0.128519,4.363889,0.133978,49.991679,51.399425,49.339386,8.124451,4.646667,0.122400,4.540000,0.133976
1,1,2017020002,2017-10-04,PPG Paints Arena,PIT,STL,2017-10-05 00:00:00,4,5,Final,0,PIT_2017-10-04,STL_2017-10-04,Matt Murray,0.941176,0.205712,0.869942,Jake Allen,0.945897,-0.138771,0.882353,PIT_2017-10-04,1.0,44.169509,52.837327,49.560

In [20]:
list(sva.columns)

['Unnamed: 0',
 'game_id',
 'date',
 'venue',
 'home_team',
 'away_team',
 'start_time',
 'home_score',
 'away_score',
 'status',
 'Home_Team_Won',
 'Home_Team_Key',
 'Away_Team_Key',
 'home_goalie',
 'home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_goalie',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%',
 'home_Team_Key',
 'home_Game_Number',
 'home_last_3_FF%_5v5',
 'home_last_3_GF%_5v5',
 'home_last_3_xGF%_5v5',
 'home_last_3_SH%',
 'home_last3_pp_TOI_per_game',
 'home_last3_xGF_per_min_pp',
 'home_last3_pk_TOI_per_game',
 'home_last3_xGA_per_min_pk',
 'home_B2B',
 'home_last_5_FF%_5v5',
 'home_last_5_GF%_5v5',
 'home_last_5_xGF%_5v5',
 'home_last_5_SH%',
 'home_last5_pp_TOI_per_game',
 'home_last5_xGF_per_min_pp',
 'home_last5_pk_TOI_per_game',
 'home_last5_xGA_per_min_pk',
 'home_last_10_FF%_5v5',
 'home_last_10_GF%_5v5',
 'home_last_10_xGF%_5v5',
 'home_last_10_SH%',
 'home_last10_pp_TOI_per_game',
 'home_last10_xGF_per_min

In [21]:
r3 = ['home_B2B', 'away_B2B', 'home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%', 'home_last_3_FF%_5v5',
 'home_last_3_GF%_5v5',
 'home_last_3_xGF%_5v5',
 'home_last_3_SH%',
 'home_last3_pp_TOI_per_game',
 'home_last3_xGF_per_min_pp',
 'home_last3_pk_TOI_per_game',
 'home_last3_xGA_per_min_pk', 'away_last_3_FF%_5v5',
 'away_last_3_GF%_5v5',
 'away_last_3_xGF%_5v5',
 'away_last_3_SH%',
 'away_last3_pp_TOI_per_game',
 'away_last3_xGF_per_min_pp',
 'away_last3_pk_TOI_per_game',
 'away_last3_xGA_per_min_pk']
r5 =['home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%', 'home_B2B', 'away_B2B', 'home_last_5_FF%_5v5',
 'home_last_5_GF%_5v5',
 'home_last_5_xGF%_5v5',
 'home_last_5_SH%',
 'home_last5_pp_TOI_per_game',
 'home_last5_xGF_per_min_pp',
 'home_last5_pk_TOI_per_game',
 'home_last5_xGA_per_min_pk', 'away_last_5_FF%_5v5',
 'away_last_5_GF%_5v5',
 'away_last_5_xGF%_5v5',
 'away_last_5_SH%',
 'away_last5_pp_TOI_per_game',
 'away_last5_xGF_per_min_pp',
 'away_last5_pk_TOI_per_game',
 'away_last5_xGA_per_min_pk']
r10 =['home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%', 'home_B2B', 'away_B2B', 'home_last_10_FF%_5v5',
 'home_last_10_GF%_5v5',
 'home_last_10_xGF%_5v5',
 'home_last_10_SH%',
 'home_last10_pp_TOI_per_game',
 'home_last10_xGF_per_min_pp',
 'home_last10_pk_TOI_per_game',
 'home_last10_xGA_per_min_pk', 'away_last_10_FF%_5v5',
 'away_last_10_GF%_5v5',
 'away_last_10_xGF%_5v5',
 'away_last_10_SH%',
 'away_last10_pp_TOI_per_game',
 'away_last10_xGF_per_min_pp',
 'away_last10_pk_TOI_per_game',
 'away_last10_xGA_per_min_pk']
r20 = ['home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%', 'home_B2B', 'away_B2B',  'home_last_20_FF%_5v5',
 'home_last_20_GF%_5v5',
 'home_last_20_xGF%_5v5',
 'home_last_20_SH%',
 'home_last20_pp_TOI_per_game',
 'home_last20_xGF_per_min_pp',
 'home_last20_pk_TOI_per_game',
 'home_last20_xGA_per_min_pk', 'away_last_20_FF%_5v5',
 'away_last_20_GF%_5v5',
 'away_last_20_xGF%_5v5',
 'away_last_20_SH%',
 'away_last20_pp_TOI_per_game',
 'away_last20_xGF_per_min_pp',
 'away_last20_pk_TOI_per_game',
 'away_last20_xGA_per_min_pk']
r30 = ['home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%', 'home_B2B', 'away_B2B',  'home_last_30_FF%_5v5',
 'home_last_30_GF%_5v5',
 'home_last_30_xGF%_5v5',
 'home_last_30_SH%',
 'home_last30_pp_TOI_per_game',
 'home_last30_xGF_per_min_pp',
 'home_last30_pk_TOI_per_game',
 'home_last30_xGA_per_min_pk', 'away_last_30_FF%_5v5',
 'away_last_30_GF%_5v5',
 'away_last_30_xGF%_5v5',
 'away_last_30_SH%',
 'away_last30_pp_TOI_per_game',
 'away_last30_xGF_per_min_pp',
 'away_last30_pk_TOI_per_game',
 'away_last30_xGA_per_min_pk']
r40 = ['home_Goalie_FenwickSV%',
 'home_Goalie_GSAx/60',
 'home_Goalie_HDCSV%',
 'away_Goalie_FenwickSV%',
 'away_Goalie_GSAx/60',
 'away_Goalie_HDCSV%', 'home_B2B', 'away_B2B',
'home_last_40_FF%_5v5',
 'home_last_40_GF%_5v5',
 'home_last_40_xGF%_5v5',
 'home_last_40_SH%',
 'home_last40_pp_TOI_per_game',
 'home_last40_xGF_per_min_pp',
 'home_last40_pk_TOI_per_game',
 'home_last40_xGA_per_min_pk',
'away_last_40_FF%_5v5',
 'away_last_40_GF%_5v5',
 'away_last_40_xGF%_5v5',
 'away_last_40_SH%',
 'away_last40_pp_TOI_per_game',
 'away_last40_xGF_per_min_pp',
 'away_last40_pk_TOI_per_game',
 'away_last40_xGA_per_min_pk']
all_r = list(set(r3+r5+r10+r20+r30))

r3_30 =list(set(r3+r30))
r5_30 = list(set(r5+r30))
r10_30 = list(set(r10+r30))
r_3_5_30 = list(set(r3+r5+r30))
r_5_20 = list(set(r5+r20))
r_5_40 = list(set(r5+r40))

In [22]:
windows =  {'3' : r3,'5' : r5,'10' : r10,'20' : r20, '30' : r30, '40' : r40, 'all' : all_r, '3+30': r3_30, '5+30': r5_30, '10+30' : r10_30, '3+5+30': r_3_5_30, '5+20': r_5_20, '5+40' : r_5_40 }

In [23]:
for k,v in windows.items():
    print(f'{k} rolling cv log loss = {model_and_evaluate(sva, v)}')

3 rolling cv log loss = -0.6842252205330045
5 rolling cv log loss = -0.6813509461758367
10 rolling cv log loss = -0.6782461857533564
20 rolling cv log loss = -0.674939527773964
30 rolling cv log loss = -0.6732347416331343
40 rolling cv log loss = -0.6713125202533432
all rolling cv log loss = -0.6866034343927327
3+30 rolling cv log loss = -0.6784814288969523
5+30 rolling cv log loss = -0.6760010240241937
10+30 rolling cv log loss = -0.6773414206948275
3+5+30 rolling cv log loss = -0.6823893282556271
5+20 rolling cv log loss = -0.6774634233379826
5+40 rolling cv log loss = -0.6745883800824078


In [24]:
for k,v in windows.items():
    print(f'{k} rolling cv log loss = {model_and_evaluate(sva, v, "accuracy")}')

3 rolling cv log loss = 0.5529139673490542
5 rolling cv log loss = 0.5664360369698541
10 rolling cv log loss = 0.5755489332296795
20 rolling cv log loss = 0.574664852725231
30 rolling cv log loss = 0.5790766174311134
40 rolling cv log loss = 0.5858361406236503
all rolling cv log loss = 0.5643867150384383
3+30 rolling cv log loss = 0.5717288589444589
5+30 rolling cv log loss = 0.5834896778094498
10+30 rolling cv log loss = 0.5767266994903689
3+5+30 rolling cv log loss = 0.5723170942385766
5+20 rolling cv log loss = 0.5761332815064352
5+40 rolling cv log loss = 0.5905402954133196


In [25]:
sva.head()

,Unnamed: 0,game_id,date,venue,home_team,away_team,start_time,home_score,away_score,status,Home_Team_Won,Home_Team_Key,Away_Team_Key,home_goalie,home_Goalie_FenwickSV%,home_Goalie_GSAx/60,home_Goalie_HDCSV%,away_goalie,away_Goalie_FenwickSV%,away_Goalie_GSAx/60,away_Goalie_HDCSV%,home_Team_Key,home_Game_Number,home_last_3_FF%_5v5,home_last_3_GF%_5v5,home_last_3_xGF%_5v5,home_last_3_SH%,home_last3_pp_TOI_per_game,home_last3_xGF_per_min_pp,home_last3_pk_TOI_per_game,home_last3_xGA_per_min_pk,home_B2B,home_last_5_FF%_5v5,home_last_5_GF%_5v5,home_last_5_xGF%_5v5,home_last_5_SH%,home_last5_pp_TOI_per_game,home_last5_xGF_per_min_pp,home_last5_pk_TOI_per_game,home_last5_xGA_per_min_pk,home_last_10_FF%_5v5,home_last_10_GF%_5v5,home_last_10_xGF%_5v5,home_last_10_SH%,home_last10_pp_TOI_per_game,home_last10_xGF_per_min_pp,home_last10_pk_TOI_per_game,home_last10_xGA_per_min_pk,home_last_20_FF%_5v5,home_last_20_GF%_5v5,home_last_20_xGF%_5v5,home_last_20_SH%,home_last20_pp_TOI_per_game,home_last20_xGF_per_min_pp,home_last20_pk_TOI_per_game,home_last20_xGA_per_min_pk,home_last_30_FF%_5v5,home_last_30_GF%_5v5,home_last_30_xGF%_5v5,home_last_30_SH%,home_last30_pp_TOI_per_game,home_last30_xGF_per_min_pp,home_last30_pk_TOI_per_game,home_last30_xGA_per_min_pk,home_last_40_FF%_5v5,home_last_40_GF%_5v5,home_last_40_xGF%_5v5,home_last_40_SH%,home_last40_pp_TOI_per_game,home_last40_xGF_per_min_pp,home_last40_pk_TOI_per_game,home_last40_xGA_per_min_pk,away_Team_Key,away_Game_Number,away_last_3_FF%_5v5,away_last_3_GF%_5v5,away_last_3_xGF%_5v5,away_last_3_SH%,away_last3_pp_TOI_per_game,away_last3_xGF_per_min_pp,away_last3_pk_TOI_per_game,away_last3_xGA_per_min_pk,away_B2B,away_last_5_FF%_5v5,away_last_5_GF%_5v5,away_last_5_xGF%_5v5,away_last_5_SH%,away_last5_pp_TOI_per_game,away_last5_xGF_per_min_pp,away_last5_pk_TOI_per_game,away_last5_xGA_per_min_pk,away_last_10_FF%_5v5,away_last_10_GF%_5v5,away_last_10_xGF%_5v5,away_last_10_SH%,away_last10_pp_TOI_per_game,away_last10_xGF_per_min_pp,away_last10_pk_TOI_per_game,away_last10_xGA_per_min_pk,away_last_20_FF%_5v5,away_last_20_GF%_5v5,away_last_20_xGF%_5v5,away_last_20_SH%,away_last20_pp_TOI_per_game,away_last20_xGF_per_min_pp,away_last20_pk_TOI_per_game,away_last20_xGA_per_min_pk,away_last_30_FF%_5v5,away_last_30_GF%_5v5,away_last_30_xGF%_5v5,away_last_30_SH%,away_last30_pp_TOI_per_game,away_last30_xGF_per_min_pp,away_last30_pk_TOI_per_game,away_last30_xGA_per_min_pk,away_last_40_FF%_5v5,away_last_40_GF%_5v5,away_last_40_xGF%_5v5,away_last_40_SH%,away_last40_pp_TOI_per_game,away_last40_xGF_per_min_pp,away_last40_pk_TOI_per_game,away_last40_xGA_per_min_pk
0,0,2017020001,2017-10-04,Bell MTS Place,WPG,TOR,2017-10-04 23:00:00,2,7,Final,0,WPG_2017-10-04,TOR_2017-10-04,Steve Mason,0.932657,-0.334940,0.866667,Frederik Andersen,0.942629,0.027934,0.872792,WPG_2017-10-04,1.0,52.073795,62.416999,48.839009,10.258649,3.444444,0.063871,4.005556,0.090707,0.0,52.399869,57.080799,51.663405,9.426112,4.190000,0.079714,3.693333,0.098556,50.977189,50.738779,51.924105,7.380972,5.128333,0.091453,4.315000,0.128158,49.296838,51.281437,51.260619,8.999958,5.860833,0.102801,5.440000,0.105423,49.171944,50.109126,49.007605,8.809652,5.171111,0.119961,4.815000,0.115057,48.803377,50.127801,48.992719,9.025236,5.328333,0.112699,4.923333,0.104858,TOR_2017-10-04,1.0,51.594385,47.355164,48.770492,8.692972,6.450000,0.106460,3.394444,0.111948,0.0,52.562502,45.937500,48.770492,6.967375,5.893333,0.069910,3.070000,0.074267,50.792085,48.572198,49.886878,7.837427,5.073333,0.080420,4.235000,0.114522,49.687136,49.188289,49.131362,7.552033,4.525833,0.118431,4.195000,0.123361,50.085902,51.013795,50.000000,7.493438,4.538889,0.128519,4.363889,0.133978,49.991679,51.399425,49.339386,8.124451,4.646667,0.122400,4.540000,0.133976
1,1,2017020002,2017-10-04,PPG Paints Arena,PIT,STL,2017-10-05 00:00:00,4,5,Final,0,PIT_2017-10-04,STL_2017-10-04,Matt Murray,0.941176,0.205712,0.869942,Jake Allen,0.945897,-0.138771,0.882353,PIT_2017-10-04,1.0,44.169509,52.837327,49.560